In [0]:
database_from = dbutils.widgets.get('database_from')
database_to = dbutils.widgets.get('database_to')

schema_name_from = dbutils.widgets.get('schema_name_from')
schema_name_to = dbutils.widgets.get('schema_name_to')

table_name_from = dbutils.widgets.get('table_name_from')
table_name_to = dbutils.widgets.get('table_name_to')

partition_field_to = dbutils.widgets.get('partition_field_to')

In [0]:
def ingestion(df):
    df.createOrReplaceTempView('source_table')
    
    with open(f'./{table_name_to}_transformation.sql', 'r', encoding='utf-8') as query_file:
        query_transformation = query_file.read() 

    query_ingestion = (f'''
            INSERT INTO {database_to}.{schema_name_to}.{table_name_to}
            REPLACE USING ({partition_field_to})
            {query_transformation}       
    ''')

    spark.sql(query_ingestion)

    return df

In [0]:
df_stream = (spark.readStream
    .option('readChangeFeed', 'true')
    .table(f'{database_from}.{schema_name_from}.{table_name_from}')
)

In [0]:
stream = (df_stream.writeStream
    .option('checkpointLocation', f'/Volumes/{database_to}/{schema_name_to}/{table_name_to}_checkpoint')
    .foreachBatch(lambda batch_df, batch_id: ingestion(batch_df))
    .trigger(availableNow = True)
)

In [0]:
stream.start()